In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

# Spark SQLContext

## Documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Initilizing SQLContext

In [2]:
!rm -rf metastore_db/
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## Creating user-defined functions

To use RDD in SQLContext, RDD lines have to be converted into a Row format

In [3]:
import re
from pyspark.sql import Row
# Read from CSV
def load_csv(line):
    return re.split("\"?,\"?", line)
        
def readInt(x):
    try:
        return int(x)
    except:
        return x

def parseElement(e):
    return Row(year=readInt(e[0]),
    month=readInt(e[1]),
    day=readInt(e[2]),
    dep_time=readInt(e[3]),
    dep_delay=readInt(e[4]),
    arr_time=readInt(e[5]),
    arr_delay=readInt(e[6]),
    cancelled=e[7],
    carrier=e[8],
    tailnum=e[9],
    flight=readInt(e[10]),
    origin=e[11],
    dest=e[12],
    air_time=readInt(e[13]),
    distance=readInt(e[14]),
    hour=readInt(e[15]),
    min=readInt(e[16])
    )

In [4]:
!wget https://dsr-data.s3.amazonaws.com/flights/flights14.csv

--2017-10-13 16:47:23--  https://dsr-data.s3.amazonaws.com/flights/flights14.csv
Resolving dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)... 52.219.73.10
Connecting to dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)|52.219.73.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16150465 (15M) [text/csv]
Saving to: ‘flights14.csv.2’

flights14.csv.2     100%[===================>]  15,40M  4,94MB/s    in 3,1s    

2017-10-13 16:47:27 (4,94 MB/s) - ‘flights14.csv.2’ saved [16150465/16150465]



In [5]:
flights = sc.textFile("flights14.csv").map(load_csv).filter(lambda e: not e[0] == "year").map(parseElement).cache()

Now the schema and the RDD have to be registered with the sqlContext:

In [6]:
flightsDF = sqlContext.createDataFrame(flights)

In [7]:
flightsDF.createOrReplaceTempView("flight")

In [8]:
sqlContext.sql("select * from flight where dest = 'LAX'").take(5)

[Row(air_time=359, arr_delay=13, arr_time=1238, cancelled='0', carrier='AA', day=1, dep_delay=14, dep_time=914, dest='LAX', distance=2475, flight=1, hour=9, min=14, month=1, origin='JFK', tailnum='N338AA', year=2014),
 Row(air_time=363, arr_delay=13, arr_time=1523, cancelled='0', carrier='AA', day=1, dep_delay=-3, dep_time=1157, dest='LAX', distance=2475, flight=3, hour=11, min=57, month=1, origin='JFK', tailnum='N335AA', year=2014),
 Row(air_time=351, arr_delay=9, arr_time=2224, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1902, dest='LAX', distance=2475, flight=21, hour=19, min=2, month=1, origin='JFK', tailnum='N327AA', year=2014),
 Row(air_time=350, arr_delay=1, arr_time=1706, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1347, dest='LAX', distance=2475, flight=117, hour=13, min=47, month=1, origin='JFK', tailnum='N319AA', year=2014),
 Row(air_time=339, arr_delay=0, arr_time=2145, cancelled='0', carrier='AA', day=1, dep_delay=4, dep_time=1824, dest='LAX', d

In [9]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").count()

2750

In [10]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").limit(2).collect()

[Row(air_time=161, arr_delay=-17, arr_time=1828, cancelled='0', carrier='AA', day=1, dep_delay=-1, dep_time=1509, dest='MIA', distance=1089, flight=145, hour=15, min=9, month=1, origin='JFK', tailnum='N5FJAA', year=2014),
 Row(air_time=166, arr_delay=-8, arr_time=1227, cancelled='0', carrier='AA', day=1, dep_delay=7, dep_time=917, dest='MIA', distance=1089, flight=1085, hour=9, min=17, month=1, origin='JFK', tailnum='N5DWAA', year=2014)]

### Count the flights that departed early and arrived late

In [14]:
flightsDF.where("dep_delay < 0 AND arr_delay > 0").limit(5).collect()

[Row(air_time=363, arr_delay=13, arr_time=1523, cancelled='0', carrier='AA', day=1, dep_delay=-3, dep_time=1157, dest='LAX', distance=2475, flight=3, hour=11, min=57, month=1, origin='JFK', tailnum='N335AA', year=2014),
 Row(air_time=232, arr_delay=1, arr_time=2036, cancelled='0', carrier='AA', day=1, dep_delay=-1, dep_time=1714, dest='AUS', distance=1521, flight=291, hour=17, min=14, month=1, origin='JFK', tailnum='N3DVAA', year=2014),
 Row(air_time=145, arr_delay=15, arr_time=940, cancelled='0', carrier='AA', day=1, dep_delay=-2, dep_time=738, dest='ORD', distance=733, flight=307, hour=7, min=38, month=1, origin='LGA', tailnum='N3GMAA', year=2014),
 Row(air_time=139, arr_delay=1, arr_time=1036, cancelled='0', carrier='AA', day=1, dep_delay=-3, dep_time=847, dest='ORD', distance=733, flight=313, hour=8, min=47, month=1, origin='LGA', tailnum='N553AA', year=2014),
 Row(air_time=145, arr_delay=24, arr_time=1349, cancelled='0', carrier='AA', day=1, dep_delay=-5, dep_time=1140, dest='ORD'

### Find the flightwith the longest arrival delay

In [23]:
sqlContext.sql("SELECT * from flight where arr_delay = (SELECT  MAX(arr_delay) FROM flight)").collect()

[Row(air_time=200, arr_delay=1494, arr_time=1008, cancelled='0', carrier='AA', day=4, dep_delay=1498, dep_time=727, dest='DFW', distance=1372, flight=1381, hour=7, min=27, month=10, origin='EWR', tailnum='N4WJAA', year=2014)]

### Find the top 10 destinations ordered by the number of flights

In [24]:
sqlContext.sql("SELECT dest, count(*) cnt from flight group by dest order by cnt desc").limit(10).collect()

[Row(dest='LAX', cnt=14434),
 Row(dest='ATL', cnt=12808),
 Row(dest='SFO', cnt=11907),
 Row(dest='MCO', cnt=11709),
 Row(dest='BOS', cnt=11609),
 Row(dest='ORD', cnt=11589),
 Row(dest='MIA', cnt=9928),
 Row(dest='CLT', cnt=9624),
 Row(dest='FLL', cnt=9471),
 Row(dest='DCA', cnt=6748)]

### Find top 10 destinations with the worst avg arrival delay, ignoring flights that arrived early

In [26]:
sqlContext.sql("SELECT dest, avg(arr_delay) avg from flight where arr_delay >= 0 group by dest order by avg desc").limit(10).collect()

[Row(dest='EGE', avg=78.03174603174604),
 Row(dest='AVP', avg=67.0),
 Row(dest='CAK', avg=55.86141304347826),
 Row(dest='TUL', avg=55.122950819672134),
 Row(dest='MSN', avg=53.32692307692308),
 Row(dest='BGR', avg=52.28813559322034),
 Row(dest='OKC', avg=51.3609022556391),
 Row(dest='IAD', avg=51.200559049615656),
 Row(dest='JAC', avg=51.142857142857146),
 Row(dest='TVC', avg=49.64705882352941)]

### Take a sample of 1% of the flights and then calculate the average departure delay for that sample

In [36]:
flightsDF.sample(False, 0.01, None).createOrReplaceTempView("flight_one")

In [37]:
sqlContext.sql("SELECT count(*) from flight").collect()

[Row(count(1)=253316)]

In [38]:
sqlContext.sql("SELECT count(*) from flight_one").collect()

[Row(count(1)=2539)]

In [39]:
sqlContext.sql("SELECT avg(dep_delay) from flight_one").collect()

[Row(avg(dep_delay)=12.461992910594722)]

### For all flights from JFK during June, show the average departure delay for each destination

In [30]:
sqlContext.sql("SELECT dest, avg(dep_delay) avg from flight where month = 6 and origin='JFK' group by dest  order by avg desc").collect()

[Row(dest='MSP', avg=28.69047619047619),
 Row(dest='SMF', avg=22.96551724137931),
 Row(dest='OAK', avg=22.75862068965517),
 Row(dest='DEN', avg=19.083333333333332),
 Row(dest='JAX', avg=18.49425287356322),
 Row(dest='CVG', avg=17.571428571428573),
 Row(dest='SAT', avg=17.17241379310345),
 Row(dest='PSE', avg=17.133333333333333),
 Row(dest='BUR', avg=16.428571428571427),
 Row(dest='SLC', avg=14.965909090909092),
 Row(dest='RSW', avg=14.913793103448276),
 Row(dest='BNA', avg=14.172413793103448),
 Row(dest='ATL', avg=13.491620111731844),
 Row(dest='IAD', avg=13.333333333333334),
 Row(dest='BOS', avg=13.006864988558352),
 Row(dest='BUF', avg=12.900826446280991),
 Row(dest='SAN', avg=12.286713286713287),
 Row(dest='SJC', avg=12.241379310344827),
 Row(dest='ORD', avg=12.136054421768707),
 Row(dest='CLT', avg=12.104803493449781),
 Row(dest='CMH', avg=11.862068965517242),
 Row(dest='SYR', avg=11.656862745098039),
 Row(dest='SEA', avg=11.639024390243902),
 Row(dest='TPA', avg=11.58235294117647)

### For every origin/dest pair, count the number of flights

In [31]:
sqlContext.sql("SELECT dest, origin, count(*) cnt from flight group by dest, origin order by cnt desc").collect()

[Row(dest='LAX', origin='JFK', cnt=10208),
 Row(dest='SFO', origin='JFK', cnt=7368),
 Row(dest='ORD', origin='LGA', cnt=7052),
 Row(dest='ATL', origin='LGA', cnt=6925),
 Row(dest='MIA', origin='LGA', cnt=5084),
 Row(dest='SFO', origin='EWR', cnt=4539),
 Row(dest='MCO', origin='JFK', cnt=4467),
 Row(dest='BOS', origin='EWR', cnt=4268),
 Row(dest='LAX', origin='EWR', cnt=4226),
 Row(dest='ATL', origin='EWR', cnt=4182),
 Row(dest='MCO', origin='EWR', cnt=4164),
 Row(dest='BOS', origin='JFK', cnt=4111),
 Row(dest='SJU', origin='JFK', cnt=4027),
 Row(dest='CLT', origin='EWR', cnt=3921),
 Row(dest='DFW', origin='LGA', cnt=3789),
 Row(dest='DCA', origin='LGA', cnt=3753),
 Row(dest='DTW', origin='LGA', cnt=3663),
 Row(dest='CLT', origin='LGA', cnt=3431),
 Row(dest='LAS', origin='JFK', cnt=3355),
 Row(dest='FLL', origin='LGA', cnt=3304),
 Row(dest='ORD', origin='EWR', cnt=3272),
 Row(dest='FLL', origin='JFK', cnt=3258),
 Row(dest='BOS', origin='LGA', cnt=3230),
 Row(dest='MCO', origin='LGA', cn